## Counting the Number of Fingers 
In this project we will be creating program that can detect a hand, segment the hand, and count the number of fingers being held up

* Firstly we will define some global variables.

* Afterwards, we'll set up a function that updates a running avearge of the background values in an ROI.

* This will later on allow us to detect new objects(hand) in the ROI.

**Startegy for Counting fingers:**
    
    Grab an ROI.
    
    Calculate a running average background value for 60 frames of video.
    
    Once avg value is found, then the hand can enter the ROI.
    
**Full Process**

1. Set an ROI and calculate the average running value for some amount of frames.

2. Then once a hand enters, we can detect a change and apply thresholding.

3. Once the hand enters the ROI, we will use a Convex hull to draw a polygon around the hand.

4. After that we will count the fingers using some maths, we'll calculate the center of the hand against the angle of outer points to inter finger counts.

In [1]:
# Import the required tools
import cv2
import numpy as np
from sklearn.metrics import pairwise

### Defining the variables and creating a background function

In [2]:
# variables
background=None   # Initial back ground is None meaning we have nothing in the background
accumulated_weight=0.5 # Our accumulated weight is between 0 and 1 i.e. 0.5
roi_top=20
roi_bottom=300
roi_right=300
roi_left=600

In [3]:
# creating function for the background 
def cal_accum_avg(frame,accumulated_weight):
    global background  # global varibale, it's initial value is none
    
    if background is None:
        background=frame.copy().astype('float')
        return None
    cv2.accumulateWeighted(frame,background,accumulated_weight)
    

### Segmentation
In this step we will use thresholding to grab the hand segment from the ROI

In [4]:
def segment(frame,threshold_min=25):
    # First calculate the absolute between the current frame and the background
    diff=cv2.absdiff(background.astype('uint8'),frame)
    
    # Apply thresholding on the absolute difference
    ret,thresholded=cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    
    # Find external contours of the hand
    image,contours,hierarchy=cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours)==0:
        return None
    
    else:
        # Assuming the largest external contours in ROI, is the hand
        hand_segment=max(contours,key=cv2.contourArea)
        
        return(thresholded,hand_segment)
    
    

### Finger Counting with Convex Hull

* Now that we have the hand segment the next step is actually count the fingers being held up.

**How Convex Hull Algo works**

* A convex hull draws a polygon by connecting point around the most external points in a frame.

* In our case, this set of points is actually just our thresholded image of hand (and the external contour information).

* First we will calculate the most extreme points (top,bottom,left and right).

* We can then calculate their intersection and estimate that as the center of the hand.

* Next we will calculate the distance for the furthest away from the center.

* Then using a ratio of that distance we create a circle.

* Any points outside of this circle and far away enough from the bottom, should be extended fingers.




In [5]:
def count_fingers(thresholded,hand_segment):
    
    conv_hull=cv2.convexHull(hand_segment)
    
    #Extreme points
    top=tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    bottom=tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    left=tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    right=tuple(conv_hull[conv_hull[:,:,0].argmax()][0])
    
    # Intersection
    cX=(left[0] + right[0])//2
    cY=(top[0] + bottom[0])//2
    
    distance=pairwise.euclidean_distances([(cX,cY)],Y=[left,right,top,bottom])[0]
    
    # Furthest point 
    max_distance=distance.max()
    
    # radius and circumference of the circle
    radius=int(0.85*max_distance)
    circumference=2*np.pi*radius
    
    # Circle roi 
    circular_roi=np.zeros(thresholded.shape[:2],dtype="uint8")
    
    # Circle
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    
    circular_roi=cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    
    image, contours, hierarchy=cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    count=0
    
    for cnt in contours:
        (x,y,w,h)=cv2.boundingRect(cnt)
        
        out_of_wrist=((cY + (cY*0.25)) > (y+h))
        
        limit_points=((circumference*0.25)>cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count +=1
            
    return count
    

### Bring it all together 

In [6]:
cam=cv2.VideoCapture(0)
num_frames=0
while True:
    ret,frame=cam.read()
    
    # Flip the frame so that it's not the mirror view
    frame=cv2.flip(frame,1)
    
    # Make current frame's copy
    frame_copy=frame.copy()
    
    # select roi
    roi=frame[roi_top:roi_bottom,roi_right:roi_left]
    
    #  change color channel of the roi
    gray=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    
    # Blur the roi to reduce noise
    gray=cv2.GaussianBlur(gray,(7,7),0)
    
    # Observe 60 frames before putting your hand in the roi 
    if num_frames<60:
        cal_accum_avg(gray,accumulated_weight)
        
        # Display a waiting text for 60 frames
        if num_frames<=59:
            cv2.putText(frame_copy,'Wait, Getting Background Avg',(200,300),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow("Finger Count",frame_copy)
    else:
        #After 60 Frames
        # calling segment function on our roi
        hand=segment(gray)
        
        # If hand is in the roi
        if hand is not None:
            thresholded,hand_segment=hand
            
            # Draw external contours of the hand
            cv2.drawContours(frame_copy,[hand_segment + (roi_right,roi_top)],-1,(255,0,0),5)
            
            # Counting Fingers
            fingers=count_fingers(thresholded,hand_segment)
            
            # Display number of fingers on the current frame
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
            # Show Thresholded 
            cv2.imshow('Threshold',thresholded)
    
    # Draw rectangle around the roi space
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    
    num_frames +=1
    
    cv2.imshow('Finger Count',frame_copy)
    k=cv2.waitKey(1)& 0xFF
    if k==27:
        break
        
cam.release()
cv2.destroyAllWindows()
            
            
            
            
            
            